# Similitud de vectores + LLM

In [44]:
# from google.colab import drive
# drive.mount('/content/drive')

Instalamos todas las librerías necesarias.

In [45]:
!pip3 install pandas scikit-learn


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# Paso 1:  Caracterización física de circuitos
Cada circuito se describe mediante características estructurales como número de curvas, tipos de curvas, longitud total de rectas, etc. A partir de ellas generamos un vector representativo para comparar con otros circuitos.


In [46]:
import pandas as pd

# Definimos las características manualmente para cada circuito
# ⚠️ Debemos mantener todas las columnas homogéneas entre circuitos
circuit_vectors = pd.DataFrame({
    "barcelona": {
        "num_curves": 16,
        "num_straights": 4,
        "slow_corners": 6,
        "medium_corners": 7,
        "fast_corners": 3,
        "longest_straight_m": 1047,
        "total_length_km": 4.657
    },
    "barein": {
        "num_curves": 15,
        "num_straights": 4,
        "slow_corners": 6,
        "medium_corners": 5,
        "fast_corners": 4,
        "longest_straight_m": 1090,
        "total_length_km": 5.412
    },
    "monaco": {
        "num_curves": 19,
        "num_straights": 2,
        "slow_corners": 11,
        "medium_corners": 7,
        "fast_corners": 1,
        "longest_straight_m": 510,
        "total_length_km": 3.337
    },
    "monza": {
        "num_curves": 11,
        "num_straights": 6,
        "slow_corners": 2,
        "medium_corners": 3,
        "fast_corners": 6,
        "longest_straight_m": 1120,
        "total_length_km": 5.793
    }
}).T  # Transponemos para que cada fila sea un circuito

circuit_vectors

,num_curves,num_straights,slow_corners,medium_corners,fast_corners,longest_straight_m,total_length_km
barcelona,16.00,4.00,6.00,7.00,3.00,1047.00,4.66
barein,15.00,4.00,6.00,5.00,4.00,1090.00,5.41
monaco,19.00,2.00,11.00,7.00,1.00,510.00,3.34
monza,11.00,6.00,2.00,3.00,6.00,1120.00,5.79


Normalizamos los vectores para poder compararlos.

In [47]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
vectors_scaled = pd.DataFrame(
    scaler.fit_transform(circuit_vectors),
    index=circuit_vectors.index,
    columns=circuit_vectors.columns
)

vectors_scaled

,num_curves,num_straights,slow_corners,medium_corners,fast_corners,longest_straight_m,total_length_km
barcelona,0.26,0.00,-0.08,0.90,-0.28,0.42,-0.15
barein,-0.09,0.00,-0.08,-0.30,0.28,0.59,0.65
monaco,1.31,-1.41,1.49,0.90,-1.39,-1.72,-1.56
monza,-1.49,1.41,-1.33,-1.51,1.39,0.71,1.06


Comparación del circuito de Monza (elegido ya que es un circuito que no comparte muchas características con ninguno de los tres circuitos).

In [48]:
from sklearn.metrics.pairwise import cosine_similarity

# Separar Monza
monza_vec = vectors_scaled.loc[["monza"]]
others_vecs = vectors_scaled.drop("monza")

# Calcular similitud coseno
similarities = cosine_similarity(monza_vec, others_vecs)[0]

# Crear DataFrame de similitud
similarity_df = pd.DataFrame({
    "circuit": others_vecs.index,
    "cosine_similarity": similarities
}).sort_values(by="cosine_similarity", ascending=False)

similarity_df

,circuit,cosine_similarity
1,barein,0.65
0,barcelona,-0.51
2,monaco,-0.94


Cargaremos los reglajes del circuito que tenga mayor similitud.

In [49]:
# Obtener el circuito más similar a Monza
most_similar_circuit = similarity_df.iloc[0]["circuit"]
print(f"El circuito más similar a Monza es: {most_similar_circuit.capitalize()}")

El circuito más similar a Monza es: Barein


Filtramos por aquellos centroides de todos circuitos donde se encuentran los setups más rápidos.

In [50]:
# Cargar los centroides de los circuitos
barcelona_kmeans_centroids = pd.read_csv("./centroids/barcelona_kmeans_centroids.csv")
barcelona_dbscan_centroids = pd.read_csv("./centroids/barcelona_dbscan_centroids.csv")

bahrein_kmeans_centroids = pd.read_csv("./centroids/bahrein_kmeans_centroids.csv")
bahrein_dbscan_centroids = pd.read_csv("./centroids/bahrein_dbscan_centroids.csv")

monaco_kmeans_centroids = pd.read_csv("./centroids/monaco_kmeans_centroids.csv")
monaco_dbscan_centroids = pd.read_csv("./centroids/monaco_dbscan_centroids.csv")

# Separar tiempos por vuelta y entradas
barcelona_kmeans_lap_times = barcelona_kmeans_centroids["lapTime"]
barcelona_dbscan_lap_times = barcelona_dbscan_centroids["lapTime"]

bahrein_kmeans_lap_times = bahrein_kmeans_centroids["lapTime"]
bahrein_dbscan_lap_times = bahrein_dbscan_centroids["lapTime"]

monaco_kmeans_lap_times = monaco_kmeans_centroids["lapTime"]
monaco_dbscan_lap_times = monaco_dbscan_centroids["lapTime"]

# Calcular la media de los tiempos por vuelta y entradas
threshold_barcelona_kmeans = barcelona_kmeans_lap_times.mean()
threshold_barcelona_dbscan = barcelona_dbscan_lap_times.mean()
print("Barcelona KMeans Threshold:", threshold_barcelona_kmeans)

threshold_bahrein_kmeans = bahrein_kmeans_lap_times.mean()
threshold_bahrein_dbscan = bahrein_dbscan_lap_times.mean()
print("Bahrein KMeans Threshold:", threshold_bahrein_kmeans)

threshold_monaco_kmeans = monaco_kmeans_lap_times.mean()
threshold_monaco_dbscan = monaco_dbscan_lap_times.mean()
print("Monaco KMeans Threshold:", threshold_monaco_kmeans)

# Filtrar por los centroides que están por debajo del umbral
barcelona_fastest_kmeans_centroids = barcelona_kmeans_centroids[barcelona_kmeans_lap_times <= threshold_barcelona_kmeans]
barcelona_fastest_dbscan_centroids = barcelona_dbscan_centroids[barcelona_dbscan_lap_times <= threshold_barcelona_dbscan]

bahrein_fastest_kmeans_centroids = bahrein_kmeans_centroids[bahrein_kmeans_lap_times <= threshold_bahrein_kmeans]
bahrein_fastest_dbscan_centroids = bahrein_dbscan_centroids[bahrein_dbscan_lap_times <= threshold_bahrein_dbscan]

monaco_fastest_kmeans_centroids = monaco_kmeans_centroids[monaco_kmeans_lap_times <= threshold_monaco_kmeans]
monaco_fastest_dbscan_centroids = monaco_dbscan_centroids[monaco_dbscan_lap_times <= threshold_monaco_dbscan]

# Mostrar los centroides más rápidos
barcelona_fastest_kmeans_setup = barcelona_fastest_kmeans_centroids.loc[barcelona_fastest_kmeans_centroids["lapTime"].idxmin()]
barcelona_fastest_kmeans_setup = barcelona_fastest_kmeans_setup.drop(["cluster", "lapTime"])
print("Barcelona KMeans Fastest Centroid:\n", barcelona_fastest_kmeans_setup)

barcelona_fastest_dbscan_centroid = barcelona_fastest_dbscan_centroids.loc[barcelona_fastest_dbscan_centroids["lapTime"].idxmin()]
barcelona_fastest_dbscan_centroid = barcelona_fastest_dbscan_centroid.drop(["cluster", "lapTime"])
print("\nBarcelona DBSCAN Fastest Centroid:\n", barcelona_fastest_dbscan_centroid)

bahrein_fastest_kmeans_centroid = bahrein_fastest_kmeans_centroids.loc[bahrein_fastest_kmeans_centroids["lapTime"].idxmin()]
bahrein_fastest_kmeans_centroid = bahrein_fastest_kmeans_centroid.drop(["cluster", "lapTime"])
print("\nBahrein KMeans Fastest Centroid:\n", bahrein_fastest_kmeans_centroid)

bahrein_fastest_dbscan_centroid = bahrein_fastest_dbscan_centroids.loc[bahrein_fastest_dbscan_centroids["lapTime"].idxmin()]
bahrein_fastest_dbscan_centroid = bahrein_fastest_dbscan_centroid.drop(["cluster", "lapTime"])
print("\nBahrein DBSCAN Fastest Centroid:\n", bahrein_fastest_dbscan_centroid)

monaco_fastest_kmeans_centroid = monaco_fastest_kmeans_centroids.loc[monaco_fastest_kmeans_centroids["lapTime"].idxmin()]
monaco_fastest_kmeans_centroid = monaco_fastest_kmeans_centroid.drop(["cluster", "lapTime"])
print("\nMonaco KMeans Fastest Centroid:\n", monaco_fastest_kmeans_centroid)

pd.set_option("display.float_format", "{:.2f}".format) # Añadimos esta opción debido al parámetro del engine braking que es casi cero en algunos casos
monaco_fastest_dbscan_centroid = monaco_fastest_dbscan_centroids.loc[monaco_fastest_dbscan_centroids["lapTime"].idxmin()]
monaco_fastest_dbscan_centroid = monaco_fastest_dbscan_centroid.drop(["cluster", "lapTime"])
print("\nMonaco DBSCAN Fastest Centroid:\n", monaco_fastest_dbscan_centroid)



Barcelona KMeans Threshold: 73587.4605984556
Bahrein KMeans Threshold: 89698.14352395725
Monaco KMeans Threshold: 71070.03017276466
Barcelona KMeans Fastest Centroid:
 m_frontWing                 44.52
m_rearWing                  30.52
m_onThrottle                25.00
m_offThrottle               20.00
m_frontCamber               -3.50
m_rearCamber                -2.20
m_frontToe                   0.00
m_rearToe                    0.00
m_frontSuspension           36.00
m_rearSuspension            11.00
m_frontAntiRollBar           4.00
m_rearAntiRollBar            2.00
m_frontSuspensionHeight     23.00
m_rearSuspensionHeight      56.00
m_brakePressure            100.00
m_brakeBias                 51.00
m_engineBraking             20.00
m_rearLeftTyrePressure      20.90
m_rearRightTyrePressure     20.90
m_frontLeftTyrePressure     23.90
m_frontRightTyrePressure    23.90
Name: 0, dtype: float64

Barcelona DBSCAN Fastest Centroid:
 m_frontWing                 46.00
m_rearWing             

## Paso 2: Recomendación de Reglajes con LLMs

Utilizaremos un modelo de lenguaje grande (LLM) ejecutado localmente mediante Ollama para generar un reglaje recomendado para un nuevo circuito (Monza), a partir de la información de setups rápidos del circuito más similar.


## Instalación y verificación de Ollama

In [51]:
# Ejecutar esto en terminal, fuera del notebook
# !ollama run llama3

In [52]:
import subprocess

# Verificar si Ollama está disponible
try:
    output = subprocess.check_output(["ollama", "list"]).decode()
    print("Modelos disponibles en Ollama:\n", output)
except Exception as e:
    print("Ollama no está disponible o no está corriendo.")

Modelos disponibles en Ollama:
 NAME                  ID              SIZE      MODIFIED    
deepseek-r1:latest    0a8c26691023    4.7 GB    6 weeks ago    
qwen3:latest          e4b5fd7f8af0    5.2 GB    6 weeks ago    
llama3:latest         365c0bd3c000    4.7 GB    6 weeks ago    



## Creación del Prompt

Convertiremos el setup del centroide en cadena de texto. 

In [53]:
SETUP_NAME_MAP = {
    "m_frontWing": "Front Wing",
    "m_rearWing": "Rear Wing",
    "m_onThrottle": "Differential On Throttle",
    "m_offThrottle": "Differential Off Throttle",
    "m_engineBraking": "Engine Braking",
    "m_frontCamber": "Front Camber",
    "m_rearCamber": "Rear Camber",
    "m_frontToe": "Front Toe",
    "m_rearToe": "Rear Toe",
    "m_frontSuspension": "Front Suspension",
    "m_rearSuspension": "Rear Suspension",
    "m_frontAntiRollBar": "Front Anti-Roll Bar",
    "m_rearAntiRollBar": "Rear Anti-Roll Bar",
    "m_frontSuspensionHeight": "Front Ride Height",
    "m_rearSuspensionHeight": "Rear Ride Height",
    "m_brakePressure": "Brake Pressure",
    "m_brakeBias": "Front Brake Bias",
    "m_rearLeftTyrePressure": "Rear Left Tyre Pressure",
    "m_rearRightTyrePressure": "Rear Right Tyre Pressure",
    "m_frontLeftTyrePressure": "Front Left Tyre Pressure",
    "m_frontRightTyrePressure": "Front Right Tyre Pressure"
}


In [54]:
import numpy as np
import pandas as pd

# ---------- Reglas oficiales del simulador ------------
SETUP_RULES = {
    "m_frontWing":            {"min": 0,    "max": 50,   "step": 1},
    "m_rearWing":             {"min": 0,    "max": 50,   "step": 1},
    "m_onThrottle":           {"min": 10,   "max": 100,  "step": 5},
    "m_offThrottle":          {"min": 10,   "max": 100,  "step": 5},
    "m_engineBraking":        {"min": 0,    "max": 100,  "step": 10},
    "m_frontCamber":          {"min": -3.5, "max": -2.5, "step": 0.1},
    "m_rearCamber":           {"min": -2.2, "max": -0.7, "step": 0.1},
    "m_frontToe":             {"min": 0.00, "max": 0.50, "step": 0.01},
    "m_rearToe":              {"min": 0.00, "max": 0.50, "step": 0.01},
    "m_frontSuspension":      {"min": 1,    "max": 41,   "step": 1},
    "m_rearSuspension":       {"min": 1,    "max": 41,   "step": 1},
    "m_frontAntiRollBar":     {"min": 1,    "max": 21,   "step": 1},
    "m_rearAntiRollBar":      {"min": 1,    "max": 21,   "step": 1},
    "m_frontSuspensionHeight":{"min": 10,   "max": 40,   "step": 1},
    "m_rearSuspensionHeight": {"min": 40,   "max": 100,  "step": 1},
    "m_brakePressure":        {"min": 80,   "max": 100,  "step": 1},
    "m_brakeBias":            {"min": 50,   "max": 70,   "step": 1},
    "m_rearLeftTyrePressure": {"min": 20.5, "max": 26.5, "step": 0.1},
    "m_rearRightTyrePressure":{"min": 20.5, "max": 26.5, "step": 0.1},
    "m_frontLeftTyrePressure":{"min": 22.5, "max": 29.5, "step": 0.1},
    "m_frontRightTyrePressure":{"min": 22.5,"max": 29.5, "step": 0.1},
}

def _quantize(value, low, high, step):
    """Recorta y redondea `value` al múltiplo de `step` más próximo dentro [low, high]."""
    value = np.clip(value, low, high)
    q = round((value - low) / step) * step + low
    # nº de decimales adecuados al tamaño del paso
    decimals = len(str(step).split(".")[1]) if isinstance(step, float) and "." in str(step) else 0
    return round(q, decimals)

def format_setup_to_dict(setup_row: pd.Series,
                         name_map: dict = SETUP_NAME_MAP,
                         rules: dict = SETUP_RULES) -> dict:
    """
    Convierte una fila de DataFrame en un dict de reglajes válidos según las reglas
    oficiales y nombres 'bonitos'. Respeta todos los pasos (0.01, 0.1, 1, 5, 10…).
    """
    temp_accumulator: dict[str, list[tuple[float, dict | None]]] = {}

    # ---- 1) cuantizamos cada valor por separado y lo acumulamos ----
    for internal_name, raw_value in setup_row.items():
        spec = rules.get(internal_name)           # puede ser None
        if spec:
            value = _quantize(raw_value,
                              spec["min"], spec["max"], spec["step"])
        else:
            value = raw_value

        pretty = name_map.get(internal_name, internal_name)
        temp_accumulator.setdefault(pretty, []).append((value, spec))

    # ---- 2) fusionamos / promediamos si hace falta y re-cuantizamos ----
    formatted: dict[str, float | int] = {}

    for pretty, tuples in temp_accumulator.items():
        values  = [v for v, _ in tuples]
        specs   = [s for _, s in tuples if s is not None]

        # Media sólo si había más de un valor (p.ej. presiones neumáticos)
        val = np.mean(values) if len(values) > 1 else values[0]

        if specs:
            # Tomamos la primera spec (todas deberían tener mismo step)
            spec0 = specs[0]
            val   = _quantize(val,
                              spec0["min"], spec0["max"], spec0["step"])
            decimals = len(str(spec0["step"]).split(".")[1]) \
                       if "." in str(spec0["step"]) else 0
        else:
            # Sin reglas: dejamos el valor tal cual, 2 decimales por defecto
            decimals = 2

        formatted[pretty] = round(val, decimals)

    return formatted




In [55]:
barcelona_kmeans_setup_dict = format_setup_to_dict(barcelona_fastest_kmeans_setup)
print("\nBarcelona KMeans Fastest Setup Dict:\n", barcelona_kmeans_setup_dict)

barcelona_dbscan_setup_dict = format_setup_to_dict(barcelona_fastest_dbscan_centroid)
print("\nBarcelona DBSCAN Fastest Setup Dict:\n", barcelona_dbscan_setup_dict)

bahrein_kmeans_setup_dict = format_setup_to_dict(bahrein_fastest_kmeans_centroid)
print("\nBahrein KMeans Fastest Setup Dict:\n", bahrein_kmeans_setup_dict)

bahrein_dbscan_setup_dict = format_setup_to_dict(bahrein_fastest_dbscan_centroid)
print("\nBahrein DBSCAN Fastest Setup Dict:\n", bahrein_dbscan_setup_dict)

monaco_kmeans_setup_dict = format_setup_to_dict(monaco_fastest_kmeans_centroid)
print("\nMonaco KMeans Fastest Setup Dict:\n", monaco_kmeans_setup_dict)

monaco_dbscan_setup_dict = format_setup_to_dict(monaco_fastest_dbscan_centroid)
print("\nMonaco DBSCAN Fastest Setup Dict:\n", monaco_dbscan_setup_dict)



Barcelona KMeans Fastest Setup Dict:
 {'Front Wing': 45, 'Rear Wing': 31, 'Differential On Throttle': 25, 'Differential Off Throttle': 20, 'Front Camber': -3.5, 'Rear Camber': -2.2, 'Front Toe': 0.0, 'Rear Toe': 0.0, 'Front Suspension': 36, 'Rear Suspension': 11, 'Front Anti-Roll Bar': 4, 'Rear Anti-Roll Bar': 2, 'Front Ride Height': 23, 'Rear Ride Height': 56, 'Brake Pressure': 100, 'Front Brake Bias': 51, 'Engine Braking': 20, 'Rear Left Tyre Pressure': 20.9, 'Rear Right Tyre Pressure': 20.9, 'Front Left Tyre Pressure': 23.9, 'Front Right Tyre Pressure': 23.9}

Barcelona DBSCAN Fastest Setup Dict:
 {'Front Wing': 46, 'Rear Wing': 39, 'Differential On Throttle': 85, 'Differential Off Throttle': 35, 'Front Camber': -3.5, 'Rear Camber': -2.2, 'Front Toe': 0.0, 'Rear Toe': 0.02, 'Front Suspension': 41, 'Rear Suspension': 6, 'Front Anti-Roll Bar': 17, 'Rear Anti-Roll Bar': 17, 'Front Ride Height': 23, 'Rear Ride Height': 57, 'Brake Pressure': 100, 'Front Brake Bias': 51, 'Engine Braking'

Indicamos los rangos válidos para los reglajes, en acorde a EA F1 24.

In [56]:
def convert_setup_to_string(setup_dict):
    return "\n".join([f"{key}: {value}" for key, value in setup_dict.items()])

# Example usage
barcelona_kmeans_setup_string = convert_setup_to_string(barcelona_kmeans_setup_dict)
print("\nBarcelona KMeans Fastest Setup String:\n", barcelona_kmeans_setup_string)

barcelona_dbscan_setup_string = convert_setup_to_string(barcelona_dbscan_setup_dict)
print("\nBarcelona DBSCAN Fastest Setup String:\n", barcelona_dbscan_setup_string)

bahrein_kmeans_setup_string = convert_setup_to_string(bahrein_kmeans_setup_dict)
print("\nBahrein KMeans Fastest Setup String:\n", bahrein_kmeans_setup_string)

bahrein_dbscan_setup_string = convert_setup_to_string(bahrein_dbscan_setup_dict)
print("\nBahrein DBSCAN Fastest Setup String:\n", bahrein_dbscan_setup_string)

monaco_kmeans_setup_string = convert_setup_to_string(monaco_kmeans_setup_dict)
print("\nMonaco KMeans Fastest Setup String:\n", monaco_kmeans_setup_string)

monaco_dbscan_setup_string = convert_setup_to_string(monaco_dbscan_setup_dict)
print("\nMonaco DBSCAN Fastest Setup String:\n", monaco_dbscan_setup_string)


Barcelona KMeans Fastest Setup String:
 Front Wing: 45
Rear Wing: 31
Differential On Throttle: 25
Differential Off Throttle: 20
Front Camber: -3.5
Rear Camber: -2.2
Front Toe: 0.0
Rear Toe: 0.0
Front Suspension: 36
Rear Suspension: 11
Front Anti-Roll Bar: 4
Rear Anti-Roll Bar: 2
Front Ride Height: 23
Rear Ride Height: 56
Brake Pressure: 100
Front Brake Bias: 51
Engine Braking: 20
Rear Left Tyre Pressure: 20.9
Rear Right Tyre Pressure: 20.9
Front Left Tyre Pressure: 23.9
Front Right Tyre Pressure: 23.9

Barcelona DBSCAN Fastest Setup String:
 Front Wing: 46
Rear Wing: 39
Differential On Throttle: 85
Differential Off Throttle: 35
Front Camber: -3.5
Rear Camber: -2.2
Front Toe: 0.0
Rear Toe: 0.02
Front Suspension: 41
Rear Suspension: 6
Front Anti-Roll Bar: 17
Rear Anti-Roll Bar: 17
Front Ride Height: 23
Rear Ride Height: 57
Brake Pressure: 100
Front Brake Bias: 51
Engine Braking: 90
Rear Left Tyre Pressure: 26.5
Rear Right Tyre Pressure: 26.5
Front Left Tyre Pressure: 29.5
Front Right Ty

Creamos el prompt de entrada.

In [57]:
julien_summary = """
Lecciones extraídas de la entrevista con Julien Schein, piloto profesional:

- Aerodinámica:
  - A mayor carga aerodinámica (ángulos más altos de ala), mayor agarre en curvas pero mayor resistencia en rectas.
  - En condiciones de lluvia, se recomienda aumentar la carga para compensar la pérdida de agarre del asfalto.

- Diferencial:
  - On-throttle bajo mejora tracción en salida de curva. On-throttle alto genera subviraje.
  - Off-throttle bajo mejora la rotación en la entrada de curva. Off-throttle alto causa subviraje por efecto del freno motor.

- Engine Braking:
  - No siempre útil si ya se alcanza el límite de frenado con freno convencional.
  - Puede ayudar en curvas rápidas donde se desacelera levantando el pie, facilitando la rotación del coche.

- Camber:
  - Más camber negativo = más agarre en curvas rápidas, pero mayor desgaste interior de neumáticos.
  - Útil en clasificación, menos recomendable en carrera.
  - Ideal cuando la rueda queda perpendicular al asfalto en máxima carga lateral.

- Toe:
  - Afecta estabilidad y respuesta en el giro.
  - Toe-out delantero mejora entrada en curva. Toe-in trasero mejora estabilidad.

- Suspensión:
  - Más dura = más estabilidad en seco. Más blanda = mejor agarre en mojado.
  - En lluvia conviene suavizar suspensión para facilitar transferencia de peso y encontrar tracción adicional.

- Barras estabilizadoras:
  - Aumentar rigidez mejora respuesta en curva media, pero reduce tracción.
  - Suavizar barra trasera mejora tracción a la salida de curva.

- Altura del coche (Ride Height):
  - Cuanto más bajo, mejor aerodinámica y centro de gravedad (en seco).
  - En mojado se puede subir el coche para aumentar el movimiento del chasis y favorecer tracción en ausencia de diferencial.

- Frenos:
  - Presión alta mejora eficacia de frenado pero incrementa riesgo de bloqueo.
  - Más bias alante = mejor frenada en recta. Más bias atrás = mejor rotación en curva.

- Presión de neumáticos:
  - Presiones más bajas suelen mejorar el agarre, pero aumentan el desgaste y riesgo de sobrecalentamiento.
  - Depende del fabricante y temperatura del asfalto. Hay que controlar el deslizamiento para evitar sobrecalentamiento progresivo.

- Lastre y carga de combustible:
  - El lastre sirve para cumplir peso mínimo y ajustar el balance.
  - Más peso delante = más agarre delante, pero afecta rendimiento general.
  - Menos combustible = mejores tiempos, pero cambia el equilibrio del coche durante la carrera.

Consejo general: todo en el setup es un compromiso. Ganar agarre en un punto suele implicar perderlo en otro.

"""

setup_ranges = """
Condición adicional importante:
El reglaje que generes debe respetar los valores válidos definidos por el juego F1 24. No uses decimales si el valor esperado es un entero. A continuación se detallan los rangos permitidos:

- Front Wing: 1–50 (entero)
- Rear Wing: 1–50 (entero)
- Differential On Throttle: 10–100 (% entero con saltos de 5)
- Differential Off Throttle: 10–100 (% entero con saltos de 5)
- Engine Braking: 0–100 (% entero con saltos de 5)
- Front Camber: -3.50 a -2.50 (decimal con dos decimales con saltos de 0.05)
- Rear Camber: -2.20 a -0.70 (decimal con dos decimales con saltos de 0.05)
- Front Toe: 0.00 a 0.50 (decimal con dos decimales con saltos de 0.05)
- Rear Toe: 0.00 a 0.50 (decimal con dos decimales con saltos de 0.05)
- Front Suspension: 1–41 (entero)
- Rear Suspension: 1–41 (entero)
- Front Anti-Roll Bar: 1–21 (entero)
- Rear Anti-Roll Bar: 1–21 (entero)
- Front Ride Height: 10–40 (entero)
- Rear Ride Height: 40–100 (entero)
- Brake Pressure: 80–100 (% entero)
- Front Brake Bias: 50–70 (% entero)
- Front Tyre Pressure: 22.5–29.5 psi (con un decimal)
- Rear Tyre Pressure: 20.5–26.5 psi (con un decimal)

Cualquier valor fuera de estos rangos será considerado inválido. No incluyas texto adicional, solo el reglaje final en el formato clave: valor.
"""

In [58]:
setup_prompt = f"""
Actúas como ingeniero de Fórmula 1. Vas a generar un reglaje base para el circuito de Monza en el videojuego de EA SPORTS F1 24.

Ten en cuenta estas lecciones de un piloto profesional sobre cómo configurar el coche según condiciones, clima y tipo de curva:
{julien_summary}

También se te proporcionan los setups más rápidos de tres circuitos previos. Usa esta información para identificar patrones comunes o útiles:

Barcelona:
{barcelona_kmeans_setup_string}

Baréin:
{bahrein_kmeans_setup_string}

Mónaco:
{monaco_kmeans_setup_string}

Con base en las características físicas del circuito de Monza, ha sido clasificado como más similar a:
{most_similar_circuit.upper()}

Ten en cuenta estas condiciones basadas en F1 24:
- Usa solo valores dentro de los rangos válidos del juego.
- Redondea a los saltos correctos:
    - Enteros donde se espera entero.
    - Un decimal donde se espera uno (ej. presión de neumáticos).
    - Dos decimales donde se requieren dos (ej. camber, toe).
    - Para valores con saltos definidos, usa el múltiplo más cercano (por ejemplo, 0.05 o 5).

A continuación, los rangos válidos por parámetro:

- Front Wing: 1–50 (entero)
- Rear Wing: 1–50 (entero)
- Differential On Throttle: 10–100 (% entero, saltos de 5)
- Differential Off Throttle: 10–100 (% entero, saltos de 5)
- Engine Braking: 0–100 (% entero, saltos de 10)
- Front Camber: -3.50 a -2.50 (dos decimales, salto 0.1)
- Rear Camber: -2.20 a -0.70 (dos decimales, salto 0.1)
- Front Toe: 0.00 a 0.50 (dos decimales, salto 0.1)
- Rear Toe: 0.00 a 0.50 (dos decimales, salto 0.1)
- Front Suspension: 1–41 (entero)
- Rear Suspension: 1–41 (entero)
- Front Anti-Roll Bars: 1–21 (entero)
- Rear Anti-Roll Bars: 1–21 (entero)
- Front Ride Height: 10–40 (entero)
- Rear Ride Height: 40–100 (entero)
- Brake Pressure: 80–100 (% entero)
- Front Brake Bias: 50–70 (% entero)
- Front Tyre Pressure: 22.5–29.5 psi (1 decimal)
- Rear Tyre Pressure: 20.5–26.5 psi (1 decimal)

Devuelve únicamente el nuevo reglaje en formato 'clave: valor' (sin texto adicional).
"""


In [59]:
print(setup_prompt)


Actúas como ingeniero de Fórmula 1. Vas a generar un reglaje base para el circuito de Monza en el videojuego de EA SPORTS F1 24.

Ten en cuenta estas lecciones de un piloto profesional sobre cómo configurar el coche según condiciones, clima y tipo de curva:

Lecciones extraídas de la entrevista con Julien Schein, piloto profesional:

- Aerodinámica:
  - A mayor carga aerodinámica (ángulos más altos de ala), mayor agarre en curvas pero mayor resistencia en rectas.
  - En condiciones de lluvia, se recomienda aumentar la carga para compensar la pérdida de agarre del asfalto.

- Diferencial:
  - On-throttle bajo mejora tracción en salida de curva. On-throttle alto genera subviraje.
  - Off-throttle bajo mejora la rotación en la entrada de curva. Off-throttle alto causa subviraje por efecto del freno motor.

- Engine Braking:
  - No siempre útil si ya se alcanza el límite de frenado con freno convencional.
  - Puede ayudar en curvas rápidas donde se desacelera levantando el pie, facilitand

## Llamadas al Modelo
Hacemos una query a los modelos LLama 3, Qwen 3 y Deepseek-R1

In [60]:
import subprocess

def query_ollama(prompt, model="llama3"):
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode(),
        stdout=subprocess.PIPE
    )
    return result.stdout.decode()

response = query_ollama(setup_prompt)
print("Respuesta del modelo:\n")
print(response)

⠙ ⠙ ⠸ ⠸ ⠴ ⠦ ⠧ ⠇ ⠇ ⠋ ⠋ ⠙ ⠸ ⠼ ⠼ ⠦ ⠦ ⠧ ⠏ ⠋ ⠋ ⠙ ⠸ ⠼ ⠼ ⠦ ⠦ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠦ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠼ ⠦ ⠧ ⠇ ⠇ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠏ ⠋ ⠋ ⠙ ⠸ ⠼ ⠼ ⠴ ⠧ ⠇ ⠏ ⠋ ⠋ ⠹ ⠸ ⠸ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠙ ⠸ ⠸ ⠴ ⠴ ⠧ ⠇ ⠏ ⠏ ⠙ ⠹ ⠸ ⠼ ⠼ ⠦ ⠦ ⠧ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠼ ⠦ ⠦ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠸ ⠴ ⠦ ⠦ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠴ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠹ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠙ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠏ ⠋ ⠋ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠇ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠴ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠸ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠋ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠋ ⠹ ⠸ ⠼ ⠼ ⠦ ⠦ ⠇ ⠏ ⠋ 

Respuesta del modelo:

Basándome en las lecciones del piloto profesional y los setups más rápidos de tres circuitos previos, he generado un reglaje base para el circuito de Monza en EA SPORTS F1 24:

Front Wing: 40
Rear Wing: 35
Differential On Throttle: 30
Differential Off Throttle: 25
Front Camber: -3.0
Rear Camber: -2.0
Front Toe: 0.10
Rear Toe: 0.15
Front Suspension: 28
Rear Suspension: 14
Front Anti-Roll Bar: 5
Rear Anti-Roll Bar: 4
Front Ride Height: 25
Rear Ride Height: 58
Brake Pressure: 97
Front Brake Bias: 55
Engine Braking: 20
Front Left Tyre Pressure: 24.0
Front Right Tyre Pressure: 24.0
Rear Left Tyre Pressure: 22.5
Rear Right Tyre Pressure: 22.5

Este reglaje combina el equilibrio entre agarre y resistencia en curvas rápidas, mientras se busca la mejor tracción en salida de curva y rotación en la entrada de curva. La configuración del diferencial y los ángulos de ala están diseñados para encontrar el compromiso adecuado entre agarre y resistencia en curvas rápidas y recta

Creamos un método que genere un prompt específico para el circuito de Monza.

In [61]:
def generate_prompt_monza(setup_bcn, setup_bhr, setup_mco, similar_circuit):
    return f"""
Eres un ingeniero de Fórmula 1 y tu tarea es generar un reglaje base para el circuito de Monza.

Basado en la experiencia del piloto profesional Julien Schein, ten en cuenta:
- Aumentar carga aerodinámica mejora agarre en curva, pero penaliza velocidad punta.
- Un diferencial más abierto al acelerar (on-throttle bajo) mejora tracción; más cerrado genera subviraje.
- Freno motor (engine braking) puede usarse para mejorar rotación en curvas rápidas.
- Más camber negativo da agarre lateral pero desgasta más el neumático.
- Toe-out delantero da agilidad en giro; toe-in trasero mejora estabilidad.
- Suspensión blanda es más efectiva en lluvia; más dura es más estable en seco.
- Barras estabilizadoras rígidas aumentan respuesta pero reducen tracción.
- Ride height más bajo mejora aerodinámica, más alto mejora tracción en mojado.
- Mayor presión de freno = más potencia pero más riesgo de bloqueo.
- Bias delantero mejora frenada recta, trasero ayuda en rotación.
- Presiones más bajas dan agarre pero sobrecalientan más rápido.
- Todo reglaje implica compromisos.

También te proporciono los reglajes más rápidos de tres circuitos previos para que identifiques patrones útiles:

Barcelona:\n{setup_bcn}

Baréin:\n{setup_bhr}

Mónaco:\n{setup_mco}

Según sus características físicas, Monza es más similar a: {similar_circuit.capitalize()}.

Instrucciones finales importantes:
- Usa solo valores válidos definidos por F1 24.
- Redondea correctamente:
  - Enteros donde se esperan enteros.
  - Un decimal para presiones de neumáticos.
  - Dos decimales para camber y toe.
  - Usa el salto correcto (0.05 o 5) cuando aplique.

Rangos válidos:
- Front/Rear Wing: 1–50
- Differential On/Off Throttle: 10–100 (saltos de 5)
- Engine Braking: 0–100 (saltos de 5)
- Front/Rear Camber: -3.50 a -2.50 / -2.20 a -0.70 (saltos de 0.05)
- Front/Rear Toe: 0.00 a 0.50 (saltos de 0.05)
- Suspension: 1–41, Anti-Roll Bar: 1–21
- Ride Height: Front 10–40, Rear 40–100
- Brake Pressure: 80–100
- Front Brake Bias: 50–70
- Tyre Pressures: Front 22.5–29.5, Rear 20.5–26.5 (1 decimal)

Devuelve solo el nuevo reglaje en formato 'Parámetro: valor', uno por línea. No añadas explicaciones.
""".strip()

Automatización avanzada de combinaciones

In [62]:
import requests
import json
from datetime import datetime

prompts = {
    "dbscan": generate_prompt_monza(barcelona_dbscan_setup_string, bahrein_dbscan_setup_string, monaco_dbscan_setup_string, most_similar_circuit),
    "kmeans": generate_prompt_monza(bahrein_kmeans_setup_string, bahrein_kmeans_setup_string, monaco_dbscan_setup_string, most_similar_circuit)
}

print("Prompts generados para DBSCAN y KMeans:\n")
print("DBSCAN Prompt:\n", prompts["dbscan"])
print("\nKMeans Prompt:\n", prompts["kmeans"])

Prompts generados para DBSCAN y KMeans:

DBSCAN Prompt:
 Eres un ingeniero de Fórmula 1 y tu tarea es generar un reglaje base para el circuito de Monza.

Basado en la experiencia del piloto profesional Julien Schein, ten en cuenta:
- Aumentar carga aerodinámica mejora agarre en curva, pero penaliza velocidad punta.
- Un diferencial más abierto al acelerar (on-throttle bajo) mejora tracción; más cerrado genera subviraje.
- Freno motor (engine braking) puede usarse para mejorar rotación en curvas rápidas.
- Más camber negativo da agarre lateral pero desgasta más el neumático.
- Toe-out delantero da agilidad en giro; toe-in trasero mejora estabilidad.
- Suspensión blanda es más efectiva en lluvia; más dura es más estable en seco.
- Barras estabilizadoras rígidas aumentan respuesta pero reducen tracción.
- Ride height más bajo mejora aerodinámica, más alto mejora tracción en mojado.
- Mayor presión de freno = más potencia pero más riesgo de bloqueo.
- Bias delantero mejora frenada recta, t

In [ ]:
models = ["deepseek-r1:latest", "llama3", "qwen3"]
temperatures = [0.6, 0.7, 0.8, 0.9]
top_ps = [0.7, 0.8, 0.9]
repetition_penalties = [1.1, 1.2]
max_tokens = 2000

results = []

def clean_deepseek_and_qwen_output(output):
    if "<think>" in output and "</think>" in output:
        return output.split("</think>")[-1].strip()
    return output.strip()

def query_ollama(prompt, model, temperature, top_p, max_tokens, repetition_penalty):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": temperature,
            "top_p": top_p,
            "num_predict": max_tokens,
            "repetition_penalty": repetition_penalty
        }
    }
    response = requests.post(url, json=payload)
    data = response.json()
    if "response" in data:
        return data["response"].strip()
        # return clean_deepseek_and_qwen_output(data["response"]) if ("qwen" or "deepeseek-r1:latest") in model else data["response"].strip()
    return f"[ERROR] {data.get('error', 'unknown error')}"

for model in models:
    for temp in temperatures:
        for top_p in top_ps:
            for rp in repetition_penalties:
                prompt_type = "kmeans" if "llama" in model else "dbscan"
                print(f"▶ Ejecutando: {model} | temp={temp} | top_p={top_p} | penalty={rp}")
                response = query_ollama(
                    prompt=prompts[prompt_type],
                    model=model,
                    temperature=temp,
                    top_p=top_p,
                    max_tokens=max_tokens,
                    repetition_penalty=rp
                )
                results.append({
                    "model": model,
                    "temperature": temp,
                    "top_p": top_p,
                    "repetition_penalty": rp,
                    "prompt_type": prompt_type,
                    "response": response
                })

# Guardar resultados
filename = f"llm_responses_monza_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Resultados guardados en {filename}")

▶ Ejecutando: deepseek-r1:latest | temp=0.6 | top_p=0.7 | penalty=1.1
▶ Ejecutando: deepseek-r1:latest | temp=0.6 | top_p=0.7 | penalty=1.2
▶ Ejecutando: deepseek-r1:latest | temp=0.6 | top_p=0.8 | penalty=1.1
▶ Ejecutando: deepseek-r1:latest | temp=0.6 | top_p=0.8 | penalty=1.2
▶ Ejecutando: deepseek-r1:latest | temp=0.6 | top_p=0.9 | penalty=1.1
▶ Ejecutando: deepseek-r1:latest | temp=0.6 | top_p=0.9 | penalty=1.2
▶ Ejecutando: deepseek-r1:latest | temp=0.7 | top_p=0.7 | penalty=1.1
▶ Ejecutando: deepseek-r1:latest | temp=0.7 | top_p=0.7 | penalty=1.2
▶ Ejecutando: deepseek-r1:latest | temp=0.7 | top_p=0.8 | penalty=1.1
▶ Ejecutando: deepseek-r1:latest | temp=0.7 | top_p=0.8 | penalty=1.2
